## 1 Install the necessary packages

### Watson OpenScale Python SDK

In [ ]:
!pip install ibm-ai-openscale

### Scikit-learn version 1.0

In [ ]:
!pip install scikit-learn==1.0

### Watson Machine Learning Python SDK

In [ ]:
!pip install -U ibm-watson-machine-learning

Restart the Notebook after Installing the required packages. By clicking on `Kernel>Restart`

## 2. Add Dataset

Select the `Insert Pandas Dataframe` option, after selecting the below cell. Ensure the variable name is `df_data_1`

## 3. Create the Call Drop Model using Scikit-Learn

In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn import preprocessing
from sklearn import svm, metrics
from scipy import sparse

from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
import json

import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV

In [ ]:
X=df_data_1.drop(['CallDrop_prec'], axis=1)
y=df_data_1.loc[:, 'CallDrop_prec']

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)



In [ ]:
'''Add a categorical transformer to your model pipeline. 
    You will need to add a label encoder into the model pipeline before storing it into WML '''

categorical_features = ["Start_Time_MM_DD_YYYY", "Traffic", "Weather", "Start_Time_HH_MM_SS_s"]
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])


In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)])

In [ ]:
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', svm.SVC(kernel='linear'))])


In [ ]:
model = pipeline.fit(X_train,y_train)

## 4. Store, Deploy and Score your Custom WML Model

In [ ]:
from ibm_watson_machine_learning import APIClient

api_key = 'vGrtX_WAm0mDnd5ECi9a7_pxRQV551Dzrcwt_Hqz8cj1' #insert your cloud API key here
location = 'us-south'

WML_CREDENTIALS = {
 "apikey": api_key,
 "url": 'https://' + location + '.ml.cloud.ibm.com'
}

ml_client = APIClient(WML_CREDENTIALS)

In [ ]:
ml_client.spaces.list(limit=10)

In [ ]:

space_uid= '91e4ef1a-4613-446f-8055-28e94e4c1962'


In [ ]:

ml_client.set.default_space(space_uid)


In [ ]:
## Store the model on WML
sw_spec_id = ml_client.software_specifications.get_id_by_name('runtime-22.1-py3.9')
print("Software Specification ID: {}".format(sw_spec_id))

meta_props={
 ml_client.repository.ModelMetaNames.NAME: 'Tower C',
 ml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sw_spec_id,
 ml_client.repository.ModelMetaNames.TYPE: "scikit-learn_1.0",
}


published_model = ml_client.repository.store_model(pipeline,
                                             meta_props=meta_props,
                                             training_data=X_train,
                                             training_target=y_train
                                             )





In [ ]:

published_model_uid = ml_client.repository.get_model_id(published_model)


ml_client.repository.list_models()
published_model_uid

In [ ]:
## Create a Deployment for your stored model
deploy_meta = {
     ml_client.deployments.ConfigurationMetaNames.NAME: 'Tower C Deployment',
     ml_client.deployments.ConfigurationMetaNames.ONLINE: {}
 }
created_deployment = ml_client.deployments.create(published_model_uid, meta_props=deploy_meta)



In [ ]:

scoring_endpoint =  ml_client.deployments.get_scoring_href(created_deployment)
deployment_uid=created_deployment['metadata']['id']
deployment_uid


## 5. Setup your Watson Openscale Dashboard 

### 5.1 Create the Watson Openscale Client

In [ ]:
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator,BearerTokenAuthenticator

from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *
iam_authenticator = IAMAuthenticator(apikey=api_key)
ai_client = APIClient(authenticator=iam_authenticator)
ai_client.data_marts.show()

ai_client.version


In [ ]:

data_mart_id = '828a6f4f-4b9d-47ec-ba42-258493a61e97'


### 5.2 Setup the Datamart on AI OpenScale

In [ ]:
data_mart_details = ai_client.data_marts.get(data_mart_id)


### 5.3 Add your Machine Learning Provider

Watson OpenScale needs to be bound to the Watson Machine Learning instance to capture payload data into and out of the model.

Note: You can bind more than one engine instance if needed by calling wos_client.service_providers.add method. Next, you can refer to particular service provider using service_provider_id.

In [ ]:

ai_client.service_providers.show()



In [ ]:

SERVICE_PROVIDER_NAME = "Watson Machine Learning - CallDrop"
SERVICE_PROVIDER_DESCRIPTION = "Added by Telecom Call Drop notebook."
added_service_provider_result = ai_client.service_providers.add(
        name=SERVICE_PROVIDER_NAME,
        description=SERVICE_PROVIDER_DESCRIPTION,
        service_type=ServiceTypes.WATSON_MACHINE_LEARNING,
        deployment_space_id = space_uid,
        operational_space_id = "production",
        credentials=WMLCredentialsCloud(
            apikey=api_key,      ## use `apikey=IAM_TOKEN` if using IAM_TOKEN to initiate client
            url=WML_CREDENTIALS["url"],
            instance_id=None
        ),
        background_mode=False
    ).result
service_provider_id = added_service_provider_result.metadata.id


In [ ]:
ai_client.service_providers.show()

In [ ]:
asset_deployment_details = ai_client.service_providers.list_assets(data_mart_id=data_mart_id, service_provider_id=service_provider_id, deployment_space_id = space_uid).result['resources'][0]
asset_deployment_details

In [ ]:
model_asset_details_from_deployment=ai_client.service_providers.get_deployment_asset(data_mart_id=data_mart_id,service_provider_id=service_provider_id,deployment_id=deployment_uid,deployment_space_id=space_uid)
model_asset_details_from_deployment

### 5.4 Perform Initial Scoring for your Model Deployment


In [ ]:
score=X_test.tail(20)
score

In [ ]:
scoring_data=list(list(x) for x in zip(*(score[x].values.tolist() for x in score.columns)))
scoring_data

In [ ]:
fields=list(X_test.columns)
print(len(fields))
fields, scoring_data[0]

In [ ]:

job_payload = {
ml_client.deployments.ScoringMetaNames.INPUT_DATA: [{
 'values': scoring_data
}]
}
print(job_payload)

In [ ]:
scoring_response = ml_client.deployments.score(deployment_uid, job_payload)

print(scoring_response)


### 5.5 Create a new Subscription 

In [ ]:
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import ScoringEndpointRequest

print(asset_deployment_details['metadata']['url'])
subscription = ai_client.subscriptions.add(
        background_mode=False,
        data_mart_id=data_mart_id,
        service_provider_id=service_provider_id,
        asset=Asset(
            asset_id=model_asset_details_from_deployment["entity"]["asset"]["asset_id"],
            name=model_asset_details_from_deployment["entity"]["asset"]["name"],
            url=model_asset_details_from_deployment["entity"]["asset"]["url"],
            asset_type=AssetTypes.MODEL,
            input_data_type=InputDataType.STRUCTURED,
            problem_type=ProblemType.BINARY_CLASSIFICATION
        ),
        deployment=AssetDeploymentRequest(
            deployment_id=asset_deployment_details['metadata']['guid'],
            name=asset_deployment_details['entity']['name'],
            deployment_type= DeploymentTypes.ONLINE,
            url=asset_deployment_details['metadata']['url'],
            scoring_endpoint=ScoringEndpointRequest(url=scoring_endpoint) # score model without shadow deployment
        ),
        asset_properties=AssetPropertiesRequest(
            label_column='CallDrop_prec',
            probability_fields=['prediction_probability'],
            prediction_field='prediction',
            feature_fields = ["Start_Time_MM_DD_YYYY","Start_Time_HH_MM_SS_s","Weather","Traffic","Total_Calls","Call_Dropped"],
            categorical_fields = ["Start_Time_MM_DD_YYYY","Start_Time_HH_MM_SS_s","Weather","Traffic"]
        )
    ).result

In [ ]:
subscription_uid = subscription.metadata.id
subscription_uid

In [ ]:
import time

time.sleep(5)
payload_data_set_id = None
payload_data_set_id = ai_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, 
                                                target_target_id=subscription_uid, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
if payload_data_set_id is None:
    print("Payload data set not found. Please check subscription status.")
else:
    print("Payload data set id: ", payload_data_set_id)

In [ ]:
ai_client.data_sets.show()

In [ ]:
ai_client.subscriptions.show()

### 5.6 Perform Inital Payload Logging
Note: You may re-use this code snippet by modifying the request_data variable to perform payload logging after finishing the initial dashboard setup

In [ ]:
fields=list(X_test.columns)

request_data = {
    "fields": fields,
    "values": scoring_data
  }
request_data


In [ ]:

"""
request_data - input to scoring endpoint in supported by Watson OpenScale format
response_data - output from scored model in supported by Watson OpenScale format
response_time - scoring request response time [ms] (integer type)

Example:

request_data = {
    "fields": ["AGE", "SEX", "BP", "CHOLESTEROL", "NA", "K"],
    "values": [[28, "F", "LOW", "HIGH", 0.61, 0.026]]
  }

response_data = {
    "fields": ["AGE", "SEX", "BP", "CHOLESTEROL", "NA", "K", "probability", "prediction", "DRUG"],
    "values": [[28, "F", "LOW", "HIGH", 0.61, 0.026, [0.82, 0.07, 0.0, 0.05, 0.03], 0.0, "drugY"]]
  }
"""

payload_scoring = {"input_data": [request_data]}

predictions = ml_client.deployments.score(deployment_uid, payload_scoring)

print("Single record scoring result:", "\n fields:", predictions["predictions"][0]["fields"], "\n values: ", predictions["predictions"][0]["values"][0])



#### Check if WML payload logging worked else manually store payload records

In [ ]:
import uuid
from ibm_watson_openscale.supporting_classes.payload_record import PayloadRecord
time.sleep(5)
pl_records_count = ai_client.data_sets.get_records_count(payload_data_set_id)
print("Number of records in the payload logging table: {}".format(pl_records_count))
if pl_records_count == 0:
    print("Payload logging did not happen, performing explicit payload logging.")
    ai_client.data_sets.store_records(data_set_id=payload_data_set_id, request_body=[PayloadRecord(
                   scoring_id=str(uuid.uuid4()),
                   request=payload_scoring,
                   response={"fields": predictions['predictions'][0]['fields'], "values":predictions['predictions'][0]['values']},
                   response_time=460
               )],background_mode=False)
    time.sleep(5)
    pl_records_count = ai_client.data_sets.get_records_count(payload_data_set_id)
    print("Number of records in the payload logging table: {}".format(pl_records_count))

### 5.9 Setup the Fairness Monitors

The code below configures fairness monitoring for our model. It turns on monitoring for two features, _conds(Weather Condition) and Traffic for the cell tower. In each case, we must specify:
  * Which model feature to monitor
  * One or more **majority** groups, which are values of that feature that we expect to receive a higher percentage of favorable outcomes
  * One or more **minority** groups, which are values of that feature that we expect to receive a higher percentage of unfavorable outcomes
  * The threshold at which we would like OpenScale to display an alert if the fairness measurement falls below (in this case, 95%)

Additionally, we must specify which outcomes from the model are favourable outcomes, and which are unfavourable. We must also provide the number of records OpenScale will use to calculate the fairness score. In this case, OpenScale's fairness monitor will run hourly, but will not calculate a new fairness rating until at least 5 records have been added. Finally, to calculate fairness, OpenScale must perform some calculations on the training data, so we provide the dataframe containing the data.

In [ ]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_uid

)
parameters = {
    "features": [
        {"feature": "Traffic",
         "majority": ['Low'],
         "minority": ['High','Medium'],
         "threshold": 0.80
         },
        {
         "feature": "Weather",
         "majority": ['Clear'],
         "minority": ['Haze','Rain','Fog','Partly Cloudy'],
         "threshold": 0.80
         }
    ],
    "favourable_class": [0],
    "unfavourable_class": [1],
    "min_records": 50
}

fairness_monitor_details = ai_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=ai_client.monitor_definitions.MONITORS.FAIRNESS.ID,
    target=target,
    parameters=parameters).result
fairness_monitor_instance_id =fairness_monitor_details.metadata.id
fairness_monitor_instance_id


### Add some more Payload (Optional for populating your dashboard)

If you wish to add some Payload Data. Take different sections of your test dataset and send to OpenScale as shown below-

In [ ]:
score=X_test.head(50)
score

In [ ]:
scoring_data=list(list(x) for x in zip(*(score[x].values.tolist() for x in score.columns)))
scoring_data

In [ ]:
fields=list(X_test.columns)
print(len(fields))
fields, scoring_data[0]

In [ ]:
request_data = {
    "fields": fields,
    "values": scoring_data
  }
request_data

In [ ]:
"""
request_data - input to scoring endpoint in supported by Watson OpenScale format
response_data - output from scored model in supported by Watson OpenScale format
response_time - scoring request response time [ms] (integer type)

Example:

request_data = {
    "fields": ["AGE", "SEX", "BP", "CHOLESTEROL", "NA", "K"],
    "values": [[28, "F", "LOW", "HIGH", 0.61, 0.026]]
  }

response_data = {
    "fields": ["AGE", "SEX", "BP", "CHOLESTEROL", "NA", "K", "probability", "prediction", "DRUG"],
    "values": [[28, "F", "LOW", "HIGH", 0.61, 0.026, [0.82, 0.07, 0.0, 0.05, 0.03], 0.0, "drugY"]]
  }
"""



payload_scoring = {"input_data": [request_data]}

predictions = ml_client.deployments.score(deployment_uid, payload_scoring)

print("Single record scoring result:", "\n fields:", predictions["predictions"][0]["fields"], "\n values: ", predictions["predictions"][0]["values"][0])

In [ ]:
time.sleep(5)
pl_records_count = ai_client.data_sets.get_records_count(payload_data_set_id)
print("Number of records in the payload logging table: {}".format(pl_records_count))
if pl_records_count == 20:
    print("Payload logging did not happen, performing explicit payload logging.")
    ai_client.data_sets.store_records(data_set_id=payload_data_set_id, request_body=[PayloadRecord(
                   scoring_id=str(uuid.uuid4()),
                   request=payload_scoring,
                   response={"fields": predictions['predictions'][0]['fields'], "values":predictions['predictions'][0]['values']},
                   response_time=460
               )],background_mode=False)
    time.sleep(5)
    pl_records_count = ai_client.data_sets.get_records_count(payload_data_set_id)
    print("Number of records in the payload logging table: {}".format(pl_records_count))